<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/ANN_2022/Image_04/class_004_image_processing_smart_trash_bin_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
import skimage.feature
from string import digits
import seaborn as sns
import skimage

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
!gdown 1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX

Downloading...
From: https://drive.google.com/uc?id=1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX
To: /content/trash_nov_22_2018.zip
100% 42.8M/42.8M [00:00<00:00, 179MB/s]


In [4]:
FILE='trash_nov_22_2018.zip'

In [5]:
archive = zipfile.ZipFile('trash_nov_22_2018.zip', 'r')
archive.extractall()

In [6]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [7]:
Img_Size = 200
ref = 'jpg'
notref = 'met'

In [8]:
X= []  
Y =[]
n = len(img_name)
i = 0
for i in range(n):
  name = img_name[i]
  if(ref in name):
    Y.append(name)
    X.append(cv2.imread(name,0))
m = len(Y)
print(n,m)

2527 2527


In [9]:
def properties(glcm_names,img,Img_Size):
  glcm =[]
  resized = cv2.resize(img, (Img_Size,Img_Size))
  m = skimage.exposure.rescale_intensity(resized, in_range='image', out_range='dtype')
  g = skimage.feature.greycomatrix(m, [1, 2], [0, np.pi/2], levels=256,normed=True, symmetric=True)
  for x in glcm_names:
    prop = skimage.feature.greycoprops(g, x)
    glcm.append(np.max(prop))
  
  return glcm

glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

# see https://github.com/ucfilho/ANN/blob/master/ANN_2022/Image/class_001_image_processing_smart_trash_bin_012.ipynb
features =[]
for i in range(len(img_name)):
  glcm = []
  img = X[i]
  glcm = properties(glcm_names,img,Img_Size)
  
  img = cv2.imread(img_name[i])
  green_img = img[:,:,1]
  values_green = properties(glcm_names,green_img,Img_Size)

  red_img = img[:,:,0]
  values_red = properties(glcm_names,red_img,Img_Size)

  blue_img = img[:,:,0]
  values_blue = properties(glcm_names,blue_img,Img_Size)

  total= [*glcm,*values_red, *values_green, *values_blue]
  
  features.append(total)



In [10]:
lista = []
glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
names = glcm_names
for x in glcm_names:
  lista.append(x+'_r')
names = glcm_names + lista

lista = []
for x in glcm_names:
  lista.append(x+'_g')
names = names + lista

lista = []
for x in glcm_names:
  lista.append(x+'_b')
names = names + lista

print(names)

df = pd.DataFrame(features,columns=names)

['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM', 'contrast_r', 'dissimilarity_r', 'homogeneity_r', 'energy_r', 'correlation_r', 'ASM_r', 'contrast_g', 'dissimilarity_g', 'homogeneity_g', 'energy_g', 'correlation_g', 'ASM_g', 'contrast_b', 'dissimilarity_b', 'homogeneity_b', 'energy_b', 'correlation_b', 'ASM_b']


In [11]:
glcm_names

['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

In [12]:
n =len(img_name)
tipos = []
selected = 'find'
for x in img_name:
  result =''.join([i for i in x[:-4] if not i.isdigit()])
  if(result != selected):
    selected = result
    tipos.append(selected)


In [13]:
label =[]
for name in img_name:
  for idx, x in enumerate(tipos):
    if(x in name):
      label.append(idx)

In [14]:
df['target']=label

In [15]:
for i in range(6): 
  df_plot= df[(df["target"] == i)]
  n= df_plot.shape[0]
  print(tipos[i],n)

cardboard 403
glass 501
metal 410
paper 594
plastic 482
trash 137


In [16]:
y=df['target']
X=df.drop(['target'], axis='columns')

In [17]:
y = pd.get_dummies(y)
# check https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
y

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
2522,0,0,0,0,0,1
2523,0,0,0,0,0,1
2524,0,0,0,0,0,1
2525,0,0,0,0,0,1


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=1)

In [19]:
scaler = StandardScaler()
# transform data
scaler_X = scaler.fit(X_train)
X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)

In [37]:
nin = X_train.shape[1]
nout =y.shape[1]
nhidden = 30

In [38]:
import tensorflow as tf

In [39]:
# Create a `Sequential` model and add a Dense layer as the first layer.
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(nin,)))
model.add(tf.keras.layers.Dense(nhidden, activation='relu'))
# Now the model will take as input arrays of shape (None, 16)
# and output arrays of shape (None, 32).
# Note that after the first layer, you don't need to specify
# the size of the input anymore:
model.add(tf.keras.layers.Dense(nout, activation='sigmoid'))


In [40]:
X_test = np.array(X_test )
y_test = np.array(y_test)
y_test = y_test.astype(np.int32)
y_train = y_train.astype(np.int32)
X_train = X_train.reshape(-1, nin) 
X_test = X_test.reshape(-1, nin)

In [41]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [42]:
# gives us back a <keras.callbacks.History object at 0x112e61a90>
#r = model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=15, batch_size=32)
r = model.fit(X_train, y_train, epochs=1000)
print("Returned:", r)

Epoch 1/1000
53/53 [==============================] - 1s 3ms/step - loss: 1.7033 - accuracy: 0.3385
Epoch 2/1000
53/53 [==============================] - 0s 3ms/step - loss: 1.5027 - accuracy: 0.4164
Epoch 3/1000
53/53 [==============================] - 0s 3ms/step - loss: 1.4305 - accuracy: 0.4318
Epoch 4/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3853 - accuracy: 0.4448
Epoch 5/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3589 - accuracy: 0.4637
Epoch 6/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3349 - accuracy: 0.4625
Epoch 7/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3157 - accuracy: 0.4778
Epoch 8/1000
53/53 [==============================] - 0s 3ms/step - loss: 1.3012 - accuracy: 0.4796
Epoch 9/1000
53/53 [==============================] - 0s 3ms/step - loss: 1.2872 - accuracy: 0.4897
Epoch 10/1000
53/53 [==============================] - 0s 3ms/step - loss: 1.2762 - accuracy: 0.4950

In [43]:
# usa a rede neural treina em calculos para treino e teste
y_calc_train=model.predict(X_train)
y_calc_test=model.predict(X_test)

In [44]:
def retorna(y):
  for yi in y:
    vmax =0
    for idx, x in enumerate(yi):
      if(x > vmax):
        ref =idx
      for 

array([[6.97114706e-01, 6.79781139e-01, 4.22808528e-03, 9.53746080e-01,
        1.63004071e-01, 2.99303830e-02],
       [2.33687460e-02, 7.96207309e-01, 4.22160625e-02, 6.97084010e-01,
        7.15756834e-01, 1.33504301e-01],
       [4.25946712e-03, 6.20746195e-01, 3.91833991e-01, 1.70996517e-01,
        8.91314685e-01, 3.89417946e-01],
       ...,
       [1.64166391e-01, 4.59832162e-01, 4.67153877e-01, 2.11630464e-02,
        2.10538030e-01, 1.88822716e-01],
       [9.61625847e-06, 1.07797980e-03, 1.39284134e-03, 9.83908772e-01,
        8.46257806e-03, 3.63328755e-02],
       [3.51634920e-02, 7.18390346e-01, 1.01083815e-02, 6.53926313e-01,
        4.96025801e-01, 2.43498921e-01]], dtype=float32)

In [30]:
y_test[0]

array([0, 0, 0, 0, 1, 0], dtype=int32)

In [ ]:
pd.__version__

In [ ]:

# check
# y_calc = pd.DataFrame(y_calc)
#y_calc = pd.from_dummies(y_calc).values # works only version 1.5
# https://pandas.pydata.org/docs/reference/api/pandas.from_dummies.html
def categoria(y):
  y = pd.DataFrame(y).values
  label =[]
  for yi in y:
    valor = 0
    for idx, x in enumerate(yi):
      if(x == 1):
        valor = idx
    label.append(valor)
  return label

y_calc = categoria(y_calc)
y_test = categoria(y_test)

In [ ]:
confusion_matrix(y_test, y_calc)

In [ ]:
# Classification Report
print('\n\n', classification_report(y_test, y_calc, target_names=tipos))

In [ ]:
''''
Img_Size = 80 hidden = 12

               precision    recall  f1-score   support

   cardboard       0.78      0.74      0.76       133
       glass       0.51      0.41      0.45       166
       metal       0.45      0.46      0.45       135
       paper       0.58      0.68      0.63       196
     plastic       0.49      0.59      0.54       159
       trash       0.47      0.18      0.26        45

    accuracy                           0.56       834
   macro avg       0.55      0.51      0.51       834
weighted avg       0.55      0.56      0.55       834

'''

In [ ]:
'''
Img_Size = 100

               precision    recall  f1-score   support

   cardboard       0.79      0.74      0.76       133
       glass       0.45      0.40      0.42       166
       metal       0.45      0.44      0.44       135
       paper       0.59      0.66      0.62       196
     plastic       0.48      0.58      0.53       159
       trash       0.59      0.22      0.32        45

    accuracy                           0.55       834
   macro avg       0.56      0.51      0.52       834
weighted avg       0.55      0.55      0.54       834
'''

In [ ]:
'''
Img_Size = 200
               precision    recall  f1-score   support

   cardboard       0.82      0.76      0.79       133
       glass       0.46      0.46      0.46       166
       metal       0.44      0.41      0.42       135
       paper       0.61      0.69      0.65       196
     plastic       0.51      0.55      0.53       159
       trash       0.36      0.18      0.24        45

    accuracy                           0.56       834
   macro avg       0.53      0.51      0.51       834
weighted avg       0.55      0.56      0.55       834
'''

In [ ]:
# Classification Report
print('\n\n', classification_report(y_test, y_test, target_names=tipos))